# Convert H5 files to CSV for Import

In [ ]:
import numpy as np
import pandas as pd
import glob
import csv

data_files = glob.glob('/tmp/data/patents/*.h5')
for file in data_files:
    
    print("Reading {}...".format(file), end=" ")
    df = pd.read_hdf(file)
    df.columns = ['title','inventors','pub_date','classifications','n_citations',
                  'cited_by', 'patent_citations','also_published_as','location']
    df['title'] = df['title'].str.strip()
    df['classifications'] = df['classifications'].str.join(";")
    df['inventors'] = df['inventors'].str.join(";")
    df['cited_by'] = df['cited_by'].str.join(";")
    df['patent_citations'] = df['patent_citations'].str.join(";")
    df['also_published_as'] = df['also_published_as'].str.join(";")
#     df.replace("", np.nan, inplace=True)
    assert(not df.index.duplicated().any())
    
    new_file = file.replace('.h5','.csv')
    print("Writing {}...".format(new_file), end=" ")
    df.to_csv(path_or_buf=new_file, sep=',', quoting=csv.QUOTE_MINIMAL, chunksize=50000, index_label='number', na_rep='na')
    print("Done.")

In [ ]:
# # Need to do something with the ALSO published as
# sum(df['also_published_as'].str.len())


# Connect to graph

In [ ]:
import json
from py2neo import Graph
from py2neo.data import Node, Relationship

# Need to get authentication working, currently NEO4J_AUTH=none
graph = Graph("bolt://neo4j:7687")

#graph = Graph("bolt://matlaber5.media.mit.edu:7687")


#graph.delete_all()

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

In [ ]:
# # Constrain one id per Patent
# print("Creating uniqueness constraint (and also index) on Patent numbers...", end=" ", flush=True)
# query = """CREATE CONSTRAINT ON (n:Patent) ASSERT n.number IS UNIQUE;"""
# graph.run(query).evaluate()
# print("Done.")

# Add index for number
print("Creating uniqueness constraint for id...", end=" ", flush=True)
query = """CREATE CONSTRAINT ON (n:Patent) ASSERT n.id IS UNIQUE;"""
graph.run(query).evaluate()
print("Done.")

# Add index for classifications
print("Creating index for classifications...", end=" ", flush=True)
query = """CREATE INDEX ON :Patent(classifications);"""
graph.run(query).evaluate()
print("Done.")

# # Add index for inventors
# print("Creating index for inventors...", end=" ", flush=True)
# query = """CREATE INDEX ON :Patent(inventors);"""
# graph.run(query).evaluate()
# print("Done.")

# Add index for location
print("Creating index for location...", end=" ", flush=True)
query = """CREATE INDEX ON :Patent(location);"""
graph.run(query).evaluate()
print("Done.")

# Add index for pub_date
print("Creating index for pub_date...", end=" ", flush=True)
query = """CREATE INDEX ON :Patent(pub_date);"""
graph.run(query).evaluate()
print("Done.")

print("Creating uniqueness constraint for author...", end=" ", flush=True)
query = """CREATE CONSTRAINT ON (a:Author) ASSERT a.name IS UNIQUE;"""
graph.run(query).evaluate()
print("Done.")

In [ ]:
# Import patents

local_data_dir = '/tmp/data/patents/'
neo4j_data_dir = '/import/patents/'

import glob, os, time

start_time = time.time()

for _, _, files in os.walk(local_data_dir):
    for file in sorted(files):
        if file.endswith('.csv'):
            print("Importing {}...".format(file), end=" ", flush=True)
            query_start_time = time.time()
            query = """
            CALL apoc.periodic.iterate("
            CALL apoc.load.csv(
                'file://{}{}',
                {{
                 header:true,sep:',',
                 mapping:{{
                  number:{{name:'id'}},
                  patent_citations:{{name:'refs',array:true,arraySep:';'}},
                  also_published_as:{{array:true,arraySep:';'}},
                  cited_by:{{array:true,arraySep:';'}},
                  inventors:{{name:'authors',array:true,arraySep:';'}},
                  n_citations:{{name:'n_citation',type:'int',arraySep:';'}},
                  classifications:{{array:true,arraySep:';'}},
                  location:{{}}
                }}
            }}) YIELD map as row 
            RETURN row
            ",
            "CREATE (p:Patent) SET p = row
                SET p.patent_citations = [f IN p.patent_citations WHERE f <> '']
                SET p.also_published_as = [f IN p.also_published_as WHERE f <> '']
                SET p.cited_by = [f IN p.cited_by WHERE f <> '']
                SET p.authors = [f IN p.authors WHERE f <> '']
                SET p.classifications = [f IN p.classifications WHERE f <> '']
                SET p.year = toInt(head(split(p.pub_date,'-')))
            ", 
            {{batchsize:10000, iterateList:true, parallel:false}}
            );
            """.format(neo4j_data_dir, file)
#             print(query)
            graph.run(query).evaluate()
            query_end_time = time.time()
            print("Done ({:.2f} minutes).".format((query_end_time-query_start_time)/60))
            
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))

In [ ]:
print("Creating AUTHORED relationships...", end=" ")
start_time = time.time()
query = """
CALL apoc.periodic.iterate(
"MATCH (p:Patent) UNWIND p.authors AS auth  RETURN p, auth"
,
"MERGE (a:Author {name: auth}) 
MERGE (a)-[:AUTHORED {is_first_author: head(p.authors)=auth, is_last_author: last(p.authors)=auth}]->(p)"
,
{batchSize:500, iterateList:true, parallel:false})
"""

# # other option
# query = """
# CALL apoc.periodic.commit(
# "MATCH (p:Patent) 
# UNWIND p.authors AS auth
# WITH p, auth
# MERGE (a:Author {name: auth}) 
# MERGE (a)-[:AUTHORED {is_first_author: head(p.authors)=auth, is_last_author: last(p.authors)=auth}]->(p)"
# ,
# {limit:1000})
# """

#print(query)
graph.run(query).evaluate()
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))

In [ ]:
print("Adding CITES relationships...", end=" ", flush=True)
import time

start_time = time.time()
query = """
CALL apoc.periodic.iterate(
"MATCH (a:Patent) UNWIND a.cited_by AS ref RETURN a, ref",
"MATCH (b:Patent {id: ref}) 
MERGE (b)-[:CITES]->(a)",
{batchSize:200, iterateList:true, parallel:false})
"""

# query = """
# CALL apoc.periodic.commit(
# "MATCH (p:Patent) UNWIND p.cited_by AS ref
# WITH p, ref
# MATCH (b:Patent {id: ref}) 
# MERGE (b)-[:CITES]->(p)"
# ,
# {limit:1000})
# """

#print(query)
graph.run(query).evaluate()
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))

In [ ]:
n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Built a graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

# Analysis

### Run PageRank once on entire graph

In [ ]:
import pandas as pd
import time

start_time = time.time()
    
print("Running PageRank on all patents...", end=" ")
# query = """
# CALL algo.pageRank.stream(
#     'MATCH (p:Patent) RETURN id(p) as id'
#     ,
#     'MATCH (p1:Patent)-[:CITES]->(p2:Patent) RETURN id(p1) as source, id(p2) as target'
#     ,
#     {graph:'cypher', iterations:20, write:true, writeProperty:'pagerank'}
#     )
# YIELD node, score
# WITH * 
# ORDER BY score DESC
# LIMIT 50
# RETURN node.number AS number, 
#     node.title AS title, 
#     node.inventors AS inventors, 
#     node.location AS location, 
#     node.n_citations AS n_citations, 
#     node.pub_date AS pub_date, 
#     node.patent_citations AS patent_citations, 
#     node.classifications AS classifications, 
#     node.cited_by AS cited_by, 
#     score;
# """


query = """
CALL algo.pageRank(
    'MATCH (p:Patent) RETURN id(p) as id'
    ,
    'MATCH (p1:Patent)-[:CITES]->(p2:Patent) RETURN id(p1) as source, id(p2) as target'
    ,
    {graph:'cypher', iterations:30, write:true, writeProperty:'pagerank'}
    );
"""
#print(query)
df = graph.run(query).to_data_frame()
    
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))

## Most influential patents and inventors

In [ ]:
start_time = time.time()
print("Finding Patents with highest PageRank...", end=" ")
query = """
MATCH (a:Author)-[:AUTHORED]->(p:Patent)
RETURN 
a.name AS name, 
COUNT(p) AS patents,
SUM(p.n_citation) AS citations, 
SUM(p.pagerank) AS pagerank
ORDER BY pagerank DESC
LIMIT 10
"""
df_patents = graph.run(query).to_data_frame()
end_time = time.time()
print("Done in {:.2f} minutes.".format((end_time-start_time)/60))

start_time = time.time()
print("Finding Authors with highest PageRank...", end=" ")
query = """
MATCH (a:Author)-[:AUTHORED]->(p:Patent)
RETURN 
a.name AS name, 
COUNT(p) AS patents,
SUM(p.n_citation) AS citations, 
SUM(p.pagerank) AS pagerank
ORDER BY pagerank DESC
LIMIT 10
"""
df_authors = graph.run(query).to_data_frame()
end_time = time.time()
print("Done in {:.2f} minutes.".format((end_time-start_time)/60))

### Run PageRank over time

In [ ]:
# Run STREAMING PageRank on each year from 1800 to 1805

import pandas as pd
import time

start_time = time.time()
start_year, end_year, step = 1900, 2020, 5
dfs = []

for year in range(start_year, end_year+1, step):
    
    print("Considering patents from < {}\n\tRunning PageRank...".format(year), end=" ")
    query = """
    CALL algo.pageRank(
        'MATCH (a:Patent)-[:CITES]->(b:Patent) WHERE a.year < {} OR b.year < {} RETURN id(a) as id'
        ,
        'MATCH (p1:Patent)-[:CITES]->(p2:Patent) RETURN id(p1) as source, id(p2) as target'
        ,
        {{graph:'cypher', iterations:30, write:true, writeProperty:'pagerank_yr'}}
        );
    """.format(year,year)
#     print(query)
    query_start_time = time.time()
    df = graph.run(query).to_data_frame()
    print("Finished ({:.2f} minutes).".format((time.time()-start_time)/60))    
    
    print("\tGetting author impact scores...".format(year), end=" ")
    query = """
    MATCH (a:Author)-[:AUTHORED]->(p:Patent)
    WHERE p.year < {}
    RETURN 
        a.name AS name, 
        COUNT(p) AS patents,
        SUM(p.n_citation) AS citations, 
        SUM(p.pagerank_yr) AS pagerank,
        SUM(LOG(p.pagerank_yr*exp(12*(2018-p.year)/-700))) AS impact
    ORDER BY impact DESC
    LIMIT 50000
    """.format(year)
#     print(query)
    query_start_time = time.time()
    df = graph.run(query).to_data_frame()
    dfs.append(df)
    print("Finished ({:.2f} minutes).".format((time.time()-query_start_time)/60))    
    end_time = time.time()

end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))


In [ ]:
for i, year in enumerate(range(start_year, end_year+1, step)):
    dfs[i]['year'] = year 

In [ ]:
all_results = pd.concat(dfs)
all_results_path = '/tmp/data/result/patents_impact_{}-{}-{}.csv'.format(start_year, end_year,step)
all_results.set_index('name', inplace=True)

print("Writing all results to {}...".format(all_results_path), end=" ")
all_results.to_csv(path_or_buf=all_results_path, sep=",", header=True, index=True)
print("Done.")

In [ ]:
all_results.tail()

In [ ]:
import numpy as np

# Pivot
result = all_results.pivot_table(index='name', columns='year', values='pagerank')    

# Remove some inventors that appear and then dissappear in the top list

first_valid_index_vec = result.apply(lambda x: x.first_valid_index(), axis=1)
last_valid_index_vec = result.apply(lambda x: x.last_valid_index(), axis=1)

start_mask = list(first_valid_index_vec<=1970)
end_mask = list(last_valid_index_vec==end_year)
full_mask = [(a and b) for a, b in zip(start_mask, end_mask)]
cleaned_results = result.iloc[full_mask,:]

# Write out to file
file_path = '/tmp/data/result/patents_pivottable_{}-{}-{}.csv'.format(start_year, end_year,step)
print("Writing results to {}...".format(file_path), end=" ")
cleaned_results.to_csv(path_or_buf=file_path, sep=",", header=True, index=True)
print("Done.")

print("Using {:,} authors in subsequent calculations.".format(cleaned_results.shape[0]))
cleaned_results.head(n=3)

## Calculate optimal portfolio

### Format data and calculate mean and standard deviation

In [ ]:
returns = cleaned_results.pct_change(periods=1, axis=1).T
mean_returns = returns.mean()
std_returns = returns.std()
cov_matrix = returns.cov()

In [ ]:
import matplotlib.pyplot as plt

# sorted_by_std = std_returns.sort_values(axis=0, ascending=False)
# high_std_index = sorted_by_std.index[1000]
# sorted_by_mean = mean_returns.sort_values(axis=0, ascending=False)
# high_mean_index = sorted_by_mean.index[1000]
# plt.plot(returns[high_std_index],'b')
# plt.plot(returns[high_mean_index],'k')

min_val = 10e-3
returns.clip_lower(min_val, inplace=True)
returns.replace(to_replace=min_val, value=None, inplace=True)

plt.figure(figsize=(8,6))
plt.semilogy(returns)
plt.grid(True)
plt.xlabel('Year')
plt.ylabel('Impact')
plt.title('Impact mean and variance')
plt.tight_layout()
plt.savefig('Mean and Variance', dpi=100)

### MC simulation


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_portfolios = 10000
n_assets = returns.shape[1]

output = np.zeros((3,num_portfolios))
for i in range(num_portfolios):
    
    n_assets_to_include = n_assets
    risk_free = 0
    
    w = np.zeros(returns.shape[1])
    w[np.random.choice(w.shape[0],n_assets_to_include)] = np.random.random(n_assets_to_include)
    w /= np.sum(w)
    
    p_return = np.sum(mean_returns*w)
    p_stdev = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))
    
    # TODO: Convert to annual basis (currently 5 years)
    output[0,i] = p_return
    output[1,i] = p_stdev
    output[2,i] = (output[0,i] - risk_free)/output[1,i]


plt.figure(figsize=(8,6))
plt.scatter(output[1,:], output[0,:], c=output[2,:], marker='o')

plt.grid(True)
plt.xlabel('Expected Variance')
plt.ylabel('Expected Impact')
plt.colorbar(label='Sharpe Ratio')
plt.title('Monte Carlo Simulation of {:,} {}-Person Grants'.format(num_portfolios, n_assets_to_include))
plt.tight_layout()
plt.savefig('Monte Carlo Simulation', dpi=100)

### Find optimal portfolio

In [ ]:
import numpy as np
import scipy.optimize as sco



## Get top patents in Boston/MA/Cambridge

In [ ]:
import time

start_time = time.time()    
print("Getting top Boston/MA/Cambridge inventors and patents...", end=" ")

query = """
MATCH (a:Author)-[:AUTHORED]->(p:Patent)
WHERE 
    p.location =~ "(?i).*[, ]?(boston|massachusetts)[, ]?.*" OR
    p.location =~ "(?i).*[, ]?(rhode island|new hampshire)[, ]?.*" OR    
    p.location =~ "(?i).*[, ]+(ma|ri|n)[, ]+.*" 
RETURN 
    a.name AS name, 
    p.location,
    COUNT(p) AS patents,
    SUM(p.n_citation) AS citations, 
    SUM(p.pagerank) AS pagerank,
    SUM(LOG(p.pagerank*exp(12*(2018-p.year)/-700))) AS impact
ORDER BY pagerank DESC
"""
df = graph.run(query).to_data_frame()
end_time = time.time()
print("Finished all calculations in {:.2f} minutes.".format((end_time-start_time)/60))

In [ ]:
print(df[['name', 'pagerank','impact']])

# Clasify and rank patents by author gender

In [ ]:
import nltk, random
from nltk.corpus import names as nltk_names

nltk.download("names")
male_names = list(nltk_names.words('male.txt'))
female_names = list(nltk_names.words('female.txt'))

labeled_names = [({"name": name}, 'male') for name in male_names]
labeled_names.extend([({"name": name}, 'female') for name in female_names])
random.shuffle(labeled_names)
classifier = nltk.NaiveBayesClassifier.train(labeled_names)

print("Trained classifier with {:,} names ({:.2f}% male, {:.2f}% female)".format(
    len(labeled_names), 
    len(male_names)/len(labeled_names)*100, 
    len(female_names)/len(labeled_names)*100))

In [ ]:
female_inventors = []
female_prob_threshold = .75

for name,score in zip(df['name'],df['pagerank']):
    split_name = name.split(' ')
    if len(split_name)>1 and len(split_name)<4 and len(split_name[0])>1:
        first_name = split_name[0]
        if classifier.prob_classify({"name": first_name}).prob('female') > female_prob_threshold:
            female_inventors.append((name,score))
print("Found {:,} female inventors".format(len(female_inventors)))

In [ ]:
file_path = '/tmp/data/result/BostonMostImpactfulFemaleInventors.csv'

score_threshold = 1
top_female_inventors = [t for t in female_inventors if t[1]>score_threshold]
print("Filtered down to the top {:,} female inventors".format(len(top_female_inventors)))

with open(file_path,'w') as file:
    file.write('Name, Impact Score\n')
    for name,score in top_female_inventors:
        print("{}, Impact Score: {:,.2f}".format(name,score))
        file.write('{}, {:.2f}\n'.format(name, score))